In [1]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from utils import *
from exp_utils import *
from tests import *
from cd_models import *

####### Fixing Seed #######
random_seed=42
np.random.seed(random_seed)
###########################

In [2]:
values = np.arange(-.24,.24,.02) +.01
print(values, len(values))

[-0.23 -0.21 -0.19 -0.17 -0.15 -0.13 -0.11 -0.09 -0.07 -0.05 -0.03 -0.01
  0.01  0.03  0.05  0.07  0.09  0.11  0.13  0.15  0.17  0.19  0.21  0.23] 24


In [3]:
n = 5000
B=100
n_p=100 #number of simulations used to estimate power
alpha=.05
task='reg'
test=.5

In [4]:
pvals=[]

for k in tqdm(range(n_p)):
    for theta in values:
        for lamb in values:

            ### Setting-up data
            Xs, ys, Xt, yt = GenData(theta, lamb, n)
            Xs_train, Xs_test, ys_train, ys_test, Zs_train, Zs_test, \
            Xt_train, Xt_test, yt_train, yt_test, Zt_train, Zt_test = prep_data(Xs, ys, Xt, yt, test=test, task=task, random_state=random_seed)

            ###
            totshift_model = KL(boost=False, cv=None)
            totshift_model.fit(Zs_train, Zt_train)
            covshift_model = KL(boost=False, cv=None)
            covshift_model.fit(Xs_train, Xt_train)

            ###
            cd_model = cde_reg(boost=False, cv=None)
            cd_model.fit(pd.concat([Xs_train, Xt_train], axis=0), 
                         pd.concat([ys_train, yt_train], axis=0))
            
            ###
            tot=Permut(Zs_test, Zt_test, totshift_model, B=B, verbose=False)
            cov=Permut(Xs_test, Xt_test, covshift_model, B=B, verbose=False)
            conc2=CondRand(Xs_test, ys_test, Xt_test, yt_test, 
                           cd_model, totshift_model, covshift_model, B=B, verbose=False)

            ###
            pvals.append([theta,lamb,tot['pval'],conc2['pval'],cov['pval']])

100%|██████████████████████████████████████| 100/100 [6:28:20<00:00, 233.00s/it]


In [5]:
pvals=np.array(pvals)

pvals.shape

(57600, 5)

In [6]:
M=np.zeros((3,len(values),len(values)))
i=0

for theta in values:
    j=0
    for lamb in values:
        ind=np.min(pvals[:,:2]==[theta,lamb], axis=1)
        M[:,len(values)-i-1,j]=np.mean(pvals[ind,2:]<alpha, axis=0)
        j+=1
    i+=1
    
np.save('outputs/cov_conc.npy', M)